In [2]:
import pandas as pd
from collections import Counter

nric_col = 'nric'

attendance_file = 'data/nric-in-attendance.xlsx'
attendance_df = pd.read_excel(attendance_file)
attendance_df = attendance_df[[nric_col]].dropna(axis=0)
# attendance_df

response_file = './data/responses.xlsx'
response_df = pd.read_excel(response_file)
# response_df

# do nric filtering

valid_nric = attendance_df.values.flatten()
invalid_nric = [nric for nric in response_df['nric'].values if nric not in valid_nric]

response_df_valid_nric_df = response_df[response_df['nric'].isin(valid_nric)]

# Get the count of each id
id_counts = response_df[nric_col].value_counts()

# Filter to get only duplicated ids (count > 1)
duplicated_ids = id_counts[id_counts > 1]

# Convert to list of tuples
duplicated_ids_list = list(duplicated_ids.items())

response_nric_dedup_df = response_df_valid_nric_df.sort_values(by=[nric_col, 'Timestamp'])

# Drop duplicates, keeping the last occurrence (latest timestamp)
final_df = response_nric_dedup_df.drop_duplicates(subset=[nric_col], keep='last')

questions = [c for c in final_df.columns if c not in ['Timestamp', nric_col]]
questions = [[q.split('. ')[0], q] for q in questions]
question_nums = [q[0] for q in questions]
question_counts = Counter(question_nums)

single_choice_questions = [q[0] for q in question_counts.items() if q[1] == 1]
multi_choice_questions_with_counts = [q for q in question_counts.items() if q[1] != 1]

unique_questions = [[q[0], q[1].split(' [')[0]] for q in questions]
unique_questions_dict = dict()
for q_num, full_q in unique_questions:
    if q_num not in unique_questions_dict:
        unique_questions_dict[q_num] = full_q
        
single_choice_questions_string = ""
for qn in single_choice_questions:
    single_choice_questions_string += f"\n{unique_questions_dict[qn]}"
    
multi_choice_questions_string = ""
for qn in multi_choice_questions_with_counts:
    multi_choice_questions_string += f"\n{unique_questions_dict[qn[0]]}"


In [3]:
output_string = (
    '## Question Overview\n\n'
    f"The single choice questions are: {single_choice_questions_string}\n\n"
    f"The multi choice questions are: {multi_choice_questions_string}\n"
    '--------------------\n'
    '## Response Overview\n\n'
    f'{len(response_df)} Responses received\n\n'
    'Invalid NRICs:\n'
    f'{invalid_nric}\n\n'
    f'{len(response_df_valid_nric_df)} Responses received from valid NRICs\n\n'
    "Duplicated NRICs and their counts:\n"
    f'{duplicated_ids_list}\n\n'
    f'{len(final_df)} Responses received from valid NRICs (deduplicated)\n'
    '--------------------\n'
)


results_string = '## Results\n\n'
for q_num, full_q in sorted(unique_questions_dict.items(), key=lambda x: x[0]):
    results_string += full_q + '\n\n'
    if q_num in single_choice_questions:
        counts = sorted(list(final_df[[full_q]].groupby(full_q).size().items()), key=lambda x: -x[1])
        for count in counts:
            results_string += 'Option: ' + count[0] + '\n'
            results_string += 'Count: ' + str(count[1]) + '\n'
            results_string += 'Percentage: ' + str(round(100 * count[1] / len(final_df), 1)) + ' % \n\n'
        results_string += f'The majority decision is {counts[0][0]} with {counts[0][1]} votes.\n'
    elif q_num in [x[0] for x in multi_choice_questions_with_counts]:
        question_cols = [q[1] for q in questions if q[0] == q_num]
        all_vals = final_df[question_cols].values.flatten()
        vals_counter = sorted(Counter(all_vals).items(), key=lambda x: -x[1])
        for counter in vals_counter:
            results_string += 'Option: ' + counter[0] + '\n'
            results_string += 'Count: ' + str(counter[1]) + '\n'
            results_string += 'Percentage: ' + str(round(100 * counter[1] / len(final_df), 1)) + ' % \n\n'
        num_choices = [x[1] for x in multi_choice_questions_with_counts if x[0] == q_num][0]
        results = vals_counter[:num_choices]
        results = [x[0] for x in results]
        results_string += f'The top {num_choices} options with the most votes are {results}\n'
    else:
        results_string += 'ERROR\n'
    results_string += '-' * 20 + '\n'

final_string = output_string + results_string

print(final_string)


## Question Overview

The single choice questions are: 
1. question a
2. question b

The multi choice questions are: 
3. choose 3 elders
--------------------
## Response Overview

5 Responses received

Invalid NRICs:
['fakenric']

4 Responses received from valid NRICs

Duplicated NRICs and their counts:
[('s1234567a', 2)]

3 Responses received from valid NRICs (deduplicated)
--------------------
## Results

1. question a

Option: no 否
Count: 2
Percentage: 66.7 % 

Option: yes 是
Count: 1
Percentage: 33.3 % 

The majority decision is no 否 with 2 votes.
--------------------
2. question b

Option: yes
Count: 3
Percentage: 100.0 % 

The majority decision is yes with 3 votes.
--------------------
3. choose 3 elders

Option: person c
Count: 3
Percentage: 100.0 % 

Option: person a
Count: 2
Percentage: 66.7 % 

Option: person b
Count: 2
Percentage: 66.7 % 

Option: person e
Count: 1
Percentage: 33.3 % 

Option: person d
Count: 1
Percentage: 33.3 % 

The top 3 options with the most votes are ['